# Part 3: Peer-to-peer Message Behaviour Data Analysis

---

### Install Python packages (pip only)

In [10]:
pip install numpy


Note: you may need to restart the kernel to use updated packages.


### Import Python packages

In [2]:
import networkx as nx
import pandas as pd
import numpy as np

---

### Task 1 of 2

Examine the file "p2p_msg_cmt224.csv" which represents messaging behaviour between users on a messaging platform. Each row has four columns, representing a single event where a person (person_a) messaged another person (person_b) on some date (date) at some time of day (time). From this, answer the following questions:

##### Q1. Build a suitable network to represent social connections based on the messaging behaviour that took place in the first 28 days. In doing so, assume that one or more messages from one person to another represents a mutual underlying social connection (i.e., regardless of whether person_a messaged person_b, person_b messaged person_a, or both at some point). 

In [20]:
data = pd.read_csv('p2p_msg_cmt224.csv')

# Convert 'date' column to datetime format
data['date'] = pd.to_datetime(data['date'], dayfirst=True)

# Filter data for the first 28 days
start_date = data['date'].min()
end_date = start_date + pd.Timedelta(days=28)  # 28 days from the minimum date
first_28_days_data = data[(data['date'] >= start_date) & (data['date'] <= end_date)]

# Print total events (messaging events) in the first 28 days
total_events = len(first_28_days_data)
print(f"Total messaging events in the first 28 days: {total_events}")

# Initialize an undirected graph for the social network
social_network = nx.Graph()

# Iterate through each messaging event in the filtered data
for _, row in first_28_days_data.iterrows():
    person_a = row['person_a']
    person_b = row['person_b']
    
    # Add an edge between person_a and person_b (mutual connection)
    social_network.add_edge(person_a, person_b)

# Calculate number of nodes and edges in the social network
num_nodes = social_network.number_of_nodes()
num_edges = social_network.number_of_edges()

print(f"Graph with {num_nodes} nodes and {num_edges} edges.")

degree_centrality = nx.degree_centrality(social_network)


Total messaging events in the first 28 days: 26133
Graph with 1174 nodes and 6822 edges.


##### Q2. Using the largest connected component of the network constructed in Task 1, Q1. What is the mean, median and the standard deviation of the differences between the maximum degree of separation of each individual and the average distance between the individual and all others?

In [12]:
largest_component = max(nx.connected_components(social_network), key=len)
LCC = social_network.subgraph(largest_component)

# Calculate maximum degree of separation (eccentricity) for each node in the LCC
max_separation = nx.eccentricity(LCC)

# Calculate average distance (eccentricity) from each node to all others in the LCC
average_distances = {}
for node in LCC.nodes():
    distances = nx.single_source_shortest_path_length(LCC, node)
    avg_distance = np.mean(list(distances.values()))
    average_distances[node] = avg_distance

# Compute differences between maximum separation and average distance for each node
differences = [max_separation[node] - average_distances[node] for node in LCC.nodes()]

# Calculate mean, median, and standard deviation of differences
mean_difference = np.mean(differences)
median_difference = np.median(differences)
std_dev_difference = np.std(differences)

print(f"Mean of differences: {mean_difference:.2f}")
print(f"Median of differences: {median_difference:.2f}")
print(f"Standard deviation of differences: {std_dev_difference:.2f}")




Mean of differences: 2.24
Median of differences: 2.23
Standard deviation of differences: 0.40


##### Q3. Build another suitable network to represent social connections based on ALL message behaviour in the dataset. In doing so, assume that one or messages from one person to another represents a MUTUAL underlying social connection (i.e., regardless of whether person_a messaged person_b, person_b messaged person_a, or both at some point).Can the social phenomenon, ‘Triadic Closure’, be supported for the common nodes that exist in both the network created from behaviour for the first 28 days (i.e., from Task 1, Q1) and the network built from all message behaviour?

In [28]:
messages_data = pd.read_csv("p2p_msg_cmt224.csv")

# Create network from all message behavior
G_all_messages = nx.from_pandas_edgelist(messages_data, source='person_a', target='person_b', create_using=nx.Graph())

# Filter data for the first 28 days
messages_data_first_28_days = messages_data[messages_data['date'] <= '2024-02-28']  # Assuming date is in ISO format (YYYY-MM-DD)

# Create network for the first 28 days
G_first_28_days = nx.from_pandas_edgelist(messages_data_first_28_days, source='person_a', target='person_b', create_using=nx.Graph())

# Find common nodes between the two networks
common_nodes = set(G_first_28_days.nodes()).intersection(G_all_messages.nodes())

# Extract subgraphs containing only the common nodes from each network
subgraph_first_28_days = G_first_28_days.subgraph(common_nodes)
subgraph_all_messages = G_all_messages.subgraph(common_nodes)

# Calculate average clustering coefficients
clustering_first_28_days = nx.average_clustering(subgraph_first_28_days)
clustering_all_messages = nx.average_clustering(subgraph_all_messages)

# Format clustering coefficients for output
def format_value(value):
    return f"{value:.2f}"

clustering_first_28_days_formatted = format_value(clustering_first_28_days)
clustering_all_messages_formatted = format_value(clustering_all_messages)

print(f"The average clustering coefficient for the subgraph over the first 28 days is {clustering_first_28_days_formatted},")
print(f"and the average clustering coefficient for the subgraph over all message behavior is {clustering_all_messages_formatted}.")


The average clustering coefficient for the subgraph over the first 28 days is 0.08,
and the average clustering coefficient for the subgraph over all message behavior is 0.11.


##### Q4. What hypothetical, non-existent edges would need to be added to the network representing all message behaviour (i.e., from Task 1, Q3) such that a message could pass along a path from any node to any other? In doing so, aim to minimise the number of edges that would be needed as well as the longest shortest path in the network as a result.

In [37]:
components = list(nx.connected_components(G_all))

# Initialize a list to store corrected new edges
new_edges = []

# Corrected approach for connecting disconnected components
if len(components) > 1:
    for i in range(len(components) - 1):
        # Choose one node from the current component
        node_from_current_comp = next(iter(components[i]))
        # Choose one node from the next component
        node_from_next_comp = next(iter(components[i + 1]))
        # Add a new edge between nodes from different components
        new_edges.append((node_from_current_comp, node_from_next_comp))

# Format the corrected new edges
new_edges_corrected_formatted = [f"({u}, {v})" for u, v in new_edges]

# Create a formatted sentence for the corrected new edges
formatted = ", ".join(new_edges_corrected_formatted)

print(f"The corrected new edges to connect the disconnected components are: {formatted}.")

The corrected new edges to connect the disconnected components are: (1, 229), (229, 1258), (1258, 1192), (1192, 1797), (1797, 1812).


### Task 2 of 2 

Using the largest connected component of the network constructed from all data in Task 1, Q2, assume the role of an outsider with complete visibility of the network that now wishes to spread a hypothetical message such that everyone in the component would know the information it contained as quickly as possible. Assume that messages will now spread in sequential timesteps using the following mechanism. If an individual is told the message at timestep 𝑡, the individual will forward the message to all of their direct connections at timestep 𝑡+1. Individuals can therefore be told the message more than once. From this, answer the following questions:

##### Q1. If you could only select 1 individual to tell at timestep 0, what set of nodes could you select from which would result in the message being received by everyone in the fewest timesteps as possible and what would the number of timesteps be?

In [4]:
def load_graph_from_edgelist(edgelist_file):
    # Load the directed graph from the edgelist file
    graph = nx.read_edgelist(edgelist_file, create_using=nx.DiGraph())
    return graph

# Define the path to your edgelist file
edgelist_file_path = "emails_cmt224.edgelist"

# Load the graph from the edgelist file
graph = load_graph_from_edgelist(edgelist_file_path)

# Now you can proceed with the influence maximization and information spread simulation
# Load the largest connected component of the network
largest_component = max(nx.weakly_connected_components(graph), key=len)
largest_component_graph = graph.subgraph(largest_component)

# Calculate node centrality measures
degree_centrality = nx.degree_centrality(largest_component_graph)
betweenness_centrality = nx.betweenness_centrality(largest_component_graph)
eigenvector_centrality = nx.eigenvector_centrality(largest_component_graph)

# Combine centrality measures to rank nodes
node_ranking = {node: (degree_centrality[node] + betweenness_centrality[node] + eigenvector_centrality[node]) 
                for node in largest_component_graph.nodes()}

# Identify the node with the highest combined centrality score
initial_node = max(node_ranking, key=node_ranking.get)

# Simulate information spread starting from the initial node
def simulate_information_spread(graph, initial_node):
    reached_nodes = set()
    queue = [(initial_node, 0)]  # (current_node, timestep)
    max_timesteps = 0
    
    while queue:
        current_node, timestep = queue.pop(0)
        
        if current_node not in reached_nodes:
            reached_nodes.add(current_node)
            max_timesteps = max(max_timesteps, timestep)
            
            for neighbor in graph.successors(current_node):
                if neighbor not in reached_nodes:
                    queue.append((neighbor, timestep + 1))
    
    return max_timesteps

# Calculate the number of timesteps required for complete dissemination
num_timesteps = simulate_information_spread(largest_component_graph, initial_node)

print("Initial node(s) to select:", initial_node)
print("Number of timesteps for complete dissemination:", num_timesteps)

Initial node(s) to select: 160
Number of timesteps for complete dissemination: 4


##### Q2. If you had to select any 5 individuals to tell at timestep 0, can the message be received by everyone in fewer timesteps than the single individual selection in Q1? In determining your answer, use one or more appropriate network connectivity measures, rather than an exhaustive search through every combination of nodes in the network.

In [39]:
def load_graph_from_edgelist(edgelist_file):
    # Load the directed graph from the edgelist file
    graph = nx.read_edgelist(edgelist_file, create_using=nx.DiGraph())
    return graph

# Load the graph from the edgelist file
edgelist_file_path = "emails_cmt224.edgelist"
graph = load_graph_from_edgelist(edgelist_file_path)

# Function to simulate information spread from multiple initial nodes
def simulate_information_spread_multiple(graph, initial_nodes):
    reached_nodes = set()
    queue = [(node, 0) for node in initial_nodes]  # (current_node, timestep)
    max_timesteps = 0
    
    while queue:
        current_node, timestep = queue.pop(0)
        
        if current_node not in reached_nodes:
            reached_nodes.add(current_node)
            max_timesteps = max(max_timesteps, timestep)
            
            for neighbor in graph.successors(current_node):
                if neighbor not in reached_nodes:
                    queue.append((neighbor, timestep + 1))
    
    return max_timesteps

# Load the largest connected component of the network
largest_component = max(nx.weakly_connected_components(graph), key=len)
largest_component_graph = graph.subgraph(largest_component)

# Calculate node centrality measures (e.g., degree centrality)
centrality_scores = nx.degree_centrality(largest_component_graph)

# Select top 5 nodes with highest centrality scores as initial nodes
selected_nodes = sorted(centrality_scores, key=centrality_scores.get, reverse=True)[:5]

# Simulate information spread starting from the selected 5 nodes
num_timesteps_multiple = simulate_information_spread_multiple(largest_component_graph, selected_nodes)

print("Selected 5 nodes at timestep 0:", selected_nodes)
print("Number of timesteps for complete dissemination (5 nodes):", num_timesteps_multiple)


Selected 5 nodes at timestep 0: ['160', '121', '107', '86', '62']
Number of timesteps for complete dissemination (5 nodes): 4
